In [251]:
import pandas as pd 
data = pd.read_excel('IVM6311_Testing_scripts.xlsx', sheet_name='Reference')

In [252]:
data.head()

,Parameter,Unnamed: 1,UVLO,VBGR_ADJ_TRIM,OUT_TEST_FRO_VVCO,OUT_TEST_FRO_IMSB,TSDN,REF_LDO,FRO_CLOCK
0,NaN,NaN,VERIFIED/NOT VERIFIED,VERIFIED/NOT VERIFIED,VERIFIED/NOT VERIFIED,VERIFIED/NOT VERIFIED,VERIFIED/NOT VERIFIED,VERIFIED/NOT VERIFIED,VERIFIED/NOT VERIFIED
1,SL.NO,NaN,0,1,2,3,4,5,5
2,Priority,NaN,UVLO,VBGR_ADJ_TRIM,OUT_TEST_FRO_VVCO,OUT_TEST_FRO_IMSB,TSDN,REF_LDO,FRO_CLOCK
3,NaN,NaN,FORCE__VBAT__3.6V\nFORCE__SDWN__1.8V\n\n0xFE_0...,Run__Startup\nRun__Enable_Ana_Testpoint\n\n0xF...,Run__Startup ...,Run__Startup ...,Run__Startup\nRun__Enable_Ana_Testpoint\n\n0xF...,Run_Startup\nRun__Enable_Ana_Testpoint\n\n0xFE...,"Run__Startup\n\n0xFE_0x01 ""Select page 1""\n0x0..."
4,Final Test Limit,Min,NaN,1793,NaN,NaN,NaN,1796,"12,5"


In [253]:
instructions = data['UVLO'].loc[3].split('\n')

In [254]:
from dft_syntaxparser import Parser
parser = Parser()

In [255]:
procedures = pd.read_excel('IVM6311_Testing_scripts.xlsx', sheet_name='Procedure')

In [256]:
procedures.columns

Index(['Startup', 'Enable_Ana_Testpoint', 'DACPA_tourn_on_wo_calibration',
       'Procedura_Maurizio', 'Boost_Test_Default', 'DAC-PA TDM_768_IN',
       'DAC-PA TDM_768_IN_and_Path_generator', 'DAC-PA PDM_3MHz_IN',
       'DAC-PA PDM_6MHz_IN', 'DAC-PA PDM_12MHz_IN',
       'DAC-PA ANALOG_IN_DC_COUPLED', 'DAC-PA ANALOG_IN_AC_COUPLED',
       'BOOST_FORCE_VBSO19V'],
      dtype='object')

In [257]:
from SwitchMatrix.mcp2221 import MCP2221
from SwitchMatrix.mcp2317 import MCP2317
mcp = MCP2221()
mcp2317 = MCP2317(mcp=mcp)
slave_address = 0x6c
def write_device(data:{}):
    device_data = mcp.mcpRead(SlaveAddress=slave_address,data=[int(data.get('RegAddr'),16)])[0]
    # print(f'Read data {hex(device_data)} ')
    bit_width = 2**(data.get('MSB') - data.get('LSB')+1)
    # print(bit_width)
    if int(data.get('Data'),16) < bit_width :
        mask = ~((bit_width-1) << data.get('LSB'))
        # print(f'mask {hex(256+mask)}')
        device_data = (device_data & mask) | ((int(data.get('Data'),16)) << data.get('LSB'))
        # print(f'Data write {hex(device_data)}')
        mcp.mcpWrite(SlaveAddress=slave_address, data=[int(data.get('RegAddr'),16),device_data])
        written_data = mcp.mcpRead(SlaveAddress=slave_address,data=[int(data.get('RegAddr'),16)])[0]
        print(f' data written {hex(written_data)}')
    else:
        print(f'Data is outof width ')
write_device({'RegAddr': '0x00', 'MSB': 7, 'LSB': 0, 'Data': '0xF'})

 data written 0xf


In [258]:
import re 
def execute_startup():
    startup_procedure = procedures['Startup'].loc[0].split('\n')
    for instruction in startup_procedure:
        instruction = instruction.lower()
        if re.match('0x',instruction):
            reg_data = parser.extract_RegisterAddress__Instruction(instruction) 
            print(reg_data)
            write_device(reg_data)
        if re.match('Force__SDWN__1.8V'.lower(), instruction):
            input('Force 1.8V on SDWN')
            
execute_startup()

{'RegAddr': '0xfe', 'MSB': 7, 'LSB': 0, 'Data': '0x0'}
 data written 0x0
{'RegAddr': '0x0', 'MSB': 7, 'LSB': 0, 'Data': '0xf'}
 data written 0xf
{'RegAddr': '0xfe', 'MSB': 7, 'LSB': 0, 'Data': '0x1'}
 data written 0x1
{'RegAddr': '0x2f', 'MSB': 7, 'LSB': 0, 'Data': '0xaa'}
 data written 0x0
{'RegAddr': '0x2f', 'MSB': 7, 'LSB': 0, 'Data': '0xbb'}
 data written 0x1


In [259]:
def execute_Enable_Ana_Testpoint():
    startup_procedure = procedures['Enable_Ana_Testpoint'].loc[0].split('\n')
    for instruction in startup_procedure:
        instruction = instruction.lower()
        if re.match('0x',instruction):
            print(parser.extract_RegisterAddress__Instruction(instruction)) 
            reg_data = parser.extract_RegisterAddress__Instruction(instruction) 
            write_device(reg_data)
        if re.match('FORCE__SDWN__OPEN'.lower(), instruction):
            input('Force SDWN OPEN')

execute_Enable_Ana_Testpoint()

{'RegAddr': '0xfe', 'MSB': 7, 'LSB': 0, 'Data': '0x1'}
 data written 0x1
{'RegAddr': '0xf', 'MSB': 7, 'LSB': 0, 'Data': '0x88'}
 data written 0x88
{'RegAddr': '0x10', 'MSB': 7, 'LSB': 0, 'Data': '0x8'}
 data written 0x8
{'RegAddr': '0x19', 'MSB': 7, 'LSB': 0, 'Data': '0x81'}
 data written 0x81
{'RegAddr': '0x1a', 'MSB': 7, 'LSB': 0, 'Data': '0x0'}
 data written 0x0


In [260]:
import re 
instructions = data['VBGR_ADJ_TRIM'].loc[3].split('\n')
for instruction in instructions:
    instruction = instruction.lower()
    # parse the instructions
    if re.match('run',instruction):
        if re.findall('startup', instruction):
           print('Startup Procedure')
           execute_startup()
        if re.findall('enable_ana_testpoint', instruction):
            print('execute_Enable_Ana_Testpoint Procedure')
            execute_Enable_Ana_Testpoint()
    if re.match('0x',instruction):
        reg_data = parser.extract_RegisterAddress__Instruction(instruction)
        write_device(reg_data)
    if re.match('measure',instruction):
        print(parser.extract_Measure__Instruction(instruction)) 
        input('measure >')
    if re.match('trim',instruction):
        print(parser.extract_Trim__Instruction(instruction)) 
    if re.match('calculate__minerror',instruction):
        print('execute min error function')

Startup Procedure
{'RegAddr': '0xfe', 'MSB': 7, 'LSB': 0, 'Data': '0x0'}
 data written 0x0
{'RegAddr': '0x0', 'MSB': 7, 'LSB': 0, 'Data': '0xf'}
 data written 0xf
{'RegAddr': '0xfe', 'MSB': 7, 'LSB': 0, 'Data': '0x1'}
 data written 0x1
{'RegAddr': '0x2f', 'MSB': 7, 'LSB': 0, 'Data': '0xaa'}
 data written 0x1
{'RegAddr': '0x2f', 'MSB': 7, 'LSB': 0, 'Data': '0xbb'}
 data written 0x1
execute_Enable_Ana_Testpoint Procedure
{'RegAddr': '0xfe', 'MSB': 7, 'LSB': 0, 'Data': '0x1'}
 data written 0x1
{'RegAddr': '0xf', 'MSB': 7, 'LSB': 0, 'Data': '0x88'}
 data written 0x88
{'RegAddr': '0x10', 'MSB': 7, 'LSB': 0, 'Data': '0x8'}
 data written 0x8
{'RegAddr': '0x19', 'MSB': 7, 'LSB': 0, 'Data': '0x81'}
 data written 0x81
{'RegAddr': '0x1a', 'MSB': 7, 'LSB': 0, 'Data': '0x0'}
 data written 0x0
 data written 0x1
 data written 0x1
{'Signal': 'sdwn', 'Unit': 'voltage'}
{'RegAddr': '0xb0', 'MSB': '0x4', 'LSB': '0x7', 'Data': None}
execute min error function
 data written 0x0
